# 모듈 임포트

In [80]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import os
import requests
from PIL import Image
import io
import numpy as np
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# chromed driver 불러오기

In [81]:
os.chmod('/Users/jeongdaegyun/Desktop/Musinsa/selenium_crawling/chromedriver_mac_arm64/chromedriver', 0o755)

driver = webdriver.Chrome(executable_path="./chromedriver_mac_arm64/chromedriver")

# 무신사 코디맵으로 이동

In [82]:
driver.get("https://store.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page=1")
driver.implicitly_wait(3)

In [99]:

def get_sample(num):
    
    # sample이 담길 list
    data = []

    # 페이지 이동
    element = driver.find_element(By.XPATH, f'/html/body/div[3]/div[3]/form/div[4]/div/ul/li[{num}]/div[2]/a/strong')
    driver.execute_script("arguments[0].click();", element)
    time.sleep(2)

    # 스타일 추출
    style = driver.find_element(By.XPATH, "/html/body/div[3]/div[3]/div[5]/span")

    # url 추출
    url = driver.current_url
    
    # 세부 아이템 링크
    items = []
    elements = driver.find_elements(By.CLASS_NAME, "brand_item")

    item_url = [element.get_attribute("href") for element in elements]
    categories = []
    for i in item_url:
        driver.get(i)
        category = driver.find_elements(By.CLASS_NAME, "item_categories")[0]
        categories.append(category)
        driver.back()


    for i in range(len(item_url)):
        items.append({"url": item_url[i], "category": categories[i]})

    
    driver.implicitly_wait(3)
    data.append(url)
    data.append(style)
    data.append(items)
    
    return data

# 데이터 추출

In [100]:
# 추출할 페이지 분량 (페이지 당 60개)
start_page = 1
end_page = 5 

In [ ]:
samples = []
for j in range(1, page_num+1):
    driver.get(f"https://store.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page={j}")
    time.sleep(2)
    for i in range(1, 61):   
        samples.append(get_sample(i))
        driver.get(f"https://store.musinsa.com/app/codimap/lists?style_type=&tag_no=&brand=&display_cnt=60&list_kind=big&sort=date&page={j}")
        time.sleep(2) 
        print(i)